In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from hurricanes.calc import convert_lon_180_to_360, find_nearest
from glob import glob
import os
from pathlib import Path

In [9]:
# urls
glider = "ng645-20210613T0000"
extent = [-98, -80, 18, 31]
# Set main path of data and plot location
root_dir = Path.home() / "Documents"

# Paths to data sources
path_data = (root_dir / "data") # create data path
path_rtofs = (path_data / "rtofs")
path_gofs = "https://tds.hycom.org/thredds/dodsC/GLBy0.08/expt_93.0"
path_gliders = (path_data / "gliders")
path_impact = (path_data / "impact_metrics")
path_impact_calculated = path_impact / "calculated"
path_impact_model = path_impact / "models"

In [3]:
# Read glider dataframe output from erddap
glider_pickle = path_gliders / f"{glider}_data.pkl"
df = pd.read_pickle(glider_pickle)
    
df = df.reset_index()
df = df.rename({"time (UTC)": "time", "longitude (degrees_east)": "lon", "latitude (degrees_north)": "lat"}, axis=1)
df = df.set_index("time").sort_index()

t0 = df.index.min().strftime("%Y-%m-%d")
t1 = df.index.max().strftime("%Y-%m-%d")

# For profile diagnostic purposes
df = df.loc[pd.IndexSlice["2021-08-18":"2021-08-22"], :]

tdf = df.groupby(level=0).first()
glon = tdf["lon"]
glat = tdf["lat"]

In [4]:
# Create dataarrays for pointwise indexing
# https://stackoverflow.com/questions/40544846/read-multiple-coordinates-with-xarray
times = xr.DataArray(tdf.index.tolist(), dims='point')
lons_gofs = xr.DataArray(convert_lon_180_to_360(glon), dims="point")
lats = xr.DataArray(glat.values, dims='point')

In [5]:
# RTOFS
# Load in RTOFS files locally
rtofs_files = []
for date in pd.date_range(t0, t1).to_list():
    files = glob(os.path.join(path_rtofs, date.strftime('rtofs.%Y%m%d'), '*.nc'))
    for f in files:
        if f == '':
            continue
        else:
            rtofs_files.append(f)

In [6]:
rtofs_files = sorted(rtofs_files)

In [7]:
# %time rtofs = xr.open_mfdataset(sorted(rtofs_files), parallel=True)

In [8]:
# ortofs = xr.open_mfdataset(sorted(rtofs_files), parallel=True)
# ortofs

In [9]:
%time rtofs = xr.open_mfdataset(sorted(rtofs_files), concat_dim="MT", combine="nested", data_vars='minimal', coords='minimal', compat='override', parallel=True)

CPU times: user 9.41 s, sys: 3.04 s, total: 12.4 s
Wall time: 9.34 s


In [7]:
rtofs = xr.open_mfdataset(sorted(rtofs_files), concat_dim="MT", combine="nested", data_vars='minimal', coords='minimal', compat='override', parallel=True)
rtofs

<xarray.Dataset>
Dimensions:      (MT: 416, Depth: 40, Y: 1710, X: 742)
Coordinates:
  * MT           (MT) datetime64[ns] 2021-06-13T06:00:00 ... 2021-09-25
    Date         (MT) float64 dask.array<chunksize=(1,), meta=np.ndarray>
  * Depth        (Depth) float32 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
  * Y            (Y) int32 1 2 3 4 5 6 7 ... 1704 1705 1706 1707 1708 1709 1710
  * X            (X) int32 1 2 3 4 5 6 7 8 9 ... 735 736 737 738 739 740 741 742
    Latitude     (Y, X) float32 dask.array<chunksize=(1710, 742), meta=np.ndarray>
    Longitude    (Y, X) float32 dask.array<chunksize=(1710, 742), meta=np.ndarray>
Data variables:
    u            (MT, Depth, Y, X) float32 dask.array<chunksize=(1, 40, 1710, 742), meta=np.ndarray>
    v            (MT, Depth, Y, X) float32 dask.array<chunksize=(1, 40, 1710, 742), meta=np.ndarray>
    temperature  (MT, Depth, Y, X) float32 dask.array<chunksize=(1, 40, 1710, 742), meta=np.ndarray>
    salinity     (MT, Depth, Y, X) float32 dask.array<chunksize=(1, 40, 1710, 742), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       HYCOM archive file
    experiment:   92.8
    history:      archv2ncdf3z

In [8]:
rtofs = rtofs.rename({'Longitude': 'lon', 'Latitude': 'lat',
                      'MT': 'time', 'Depth': 'depth'})

In [19]:
rtofs[["lon", "lat"]].load()

<xarray.Dataset>
Dimensions:  (Y: 1710, X: 742)
Coordinates:
    lon      (Y, X) float32 -100.0 -99.92 -99.84 -99.76 ... -91.87 -91.82 -91.77
    lat      (Y, X) float32 0.0 0.0 0.0 0.0 0.0 ... 79.65 79.68 79.72 79.75
  * Y        (Y) int32 1 2 3 4 5 6 7 8 ... 1704 1705 1706 1707 1708 1709 1710
  * X        (X) int32 1 2 3 4 5 6 7 8 9 ... 734 735 736 737 738 739 740 741 742
Data variables:
    *empty*
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       HYCOM archive file
    experiment:   92.8
    history:      archv2ncdf3z

In [20]:
xds = rtofs.isel(time=0)

In [31]:
%%timeit
# Save rtofs lon and lat as variables to speed up indexing calculation
rtofs_lon = xds.lon.values
rtofs_lat = xds.lat.values

# Find index of nearest lon and lat points
_, lon1_ind = find_nearest(rtofs_lon[0, :], extent[0])
_, lon2_ind = find_nearest(rtofs_lon[0, :], extent[1])

_, lat1_ind = find_nearest(rtofs_lat[:, 0], extent[2])
_, lat2_ind = find_nearest(rtofs_lat[:, 0], extent[3])
# lon1_ind, lon2_ind, lat1_ind, lat2_ind

117 µs ± 361 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
# Find index of nearest lon and lat points
calc_lon = np.array([find_nearest(rtofs_lon[0, :], lon) for lon in glon.values])
calc_lat = np.array([find_nearest(rtofs_lat[:, 0], lat) for lat in glat.values])

In [13]:
# Create dataarrays for pointwise indexing
# https://stackoverflow.com/questions/40544846/
# read-multiple-coordinates-with-xarray
lons = xr.DataArray(calc_lon[:,1], dims='point')
lats = xr.DataArray(calc_lat[:,1], dims='point')

In [17]:
%%timeit
grtofs = rtofs[['temperature', 'salinity']].sel(time=times, Y=lats, X=lons, method="nearest")
grtofs.to_netcdf(path_impact_model / f"{glider}_rtofs_data_1.nc")

9.38 ms ± 94.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
%time grtofs.to_netcdf(path_impact_model / f"{glider}_rtofs_data_1.nc")

CPU times: user 41.2 s, sys: 5.76 s, total: 46.9 s
Wall time: 41 s


In [18]:
%%timeit
trtofs = rtofs[['temperature', 'salinity']].sel(time=times, method="nearest")
trtofs = trtofs.sel(Y=lats, X=lons)
trtofs.to_netcdf(path_impact_model / f"{glider}_rtofs_data_2.nc")

11.5 ms ± 64.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
%time trtofs.to_netcdf(path_impact_model / f"{glider}_rtofs_data_2.nc")

CPU times: user 38.6 s, sys: 6.69 s, total: 45.3 s
Wall time: 35.9 s


In [36]:
%%timeit
qrtofs = rtofs[['temperature', 'salinity']].sel(time=slice("2021-08-18", "2021-08-22"))
qrtofs = qrtofs.sel(Y=lats, X=lons)

In [38]:
%time qrtofs.to_netcdf(path_impact_model / f"{glider}_rtofs_data_3.nc")

CPU times: user 38.9 s, sys: 5.34 s, total: 44.3 s
Wall time: 38.8 s


In [21]:
ds1 = xr.open_dataset(path_impact_model / f"{glider}_rtofs_data_1.nc")
ds2 = xr.open_dataset(path_impact_model / f"{glider}_rtofs_data_2.nc")

In [23]:
ds1.identical(ds2)

True

In [27]:
ds1.close()
ds2.close()

In [37]:
qrtofs

<xarray.Dataset>
Dimensions:      (time: 20, depth: 40, point: 35)
Coordinates:
  * time         (time) datetime64[ns] 2021-08-18 ... 2021-08-22T18:00:00
    Date         (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
  * depth        (depth) float32 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
    Y            (point) int32 365 365 365 365 365 366 ... 368 368 368 368 368
    X            (point) int32 131 131 130 131 131 132 ... 143 144 144 144 144
    lat          (point) float32 dask.array<chunksize=(35,), meta=np.ndarray>
    lon          (point) float32 dask.array<chunksize=(35,), meta=np.ndarray>
Dimensions without coordinates: point
Data variables:
    temperature  (time, depth, point) float32 dask.array<chunksize=(1, 40, 35), meta=np.ndarray>
    salinity     (time, depth, point) float32 dask.array<chunksize=(1, 40, 35), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       HYCOM archive file
    experiment:   92.8
    history:      archv2ncdf3z

In [39]:
ds = xr.open_dataset(path_gofs, drop_variables="tau")

In [ ]:
ds["lon"] = convert_lon_360_to_180(ds["lon"])

In [42]:
from hurricanes.calc import convert_lon_360_to_180

In [62]:
ds["lon"] = convert_lon_360_to_180(ds["lon"])

In [66]:
ds = ds.sel(
    lon=slice(extent[0], extent[1]),
    lat=slice(extent[2], extent[3])
)
ds

<xarray.Dataset>
Dimensions:            (lat: 326, lon: 226, time: 9745, depth: 40)
Coordinates:
  * lat                (lat) float64 18.0 18.04 18.08 18.12 ... 30.92 30.96 31.0
  * lon                (lon) float64 -98.0 -97.92 -97.84 ... -80.16 -80.08 -80.0
  * time               (time) datetime64[ns] 2018-12-04T12:00:00 ... 2022-04-...
  * depth              (depth) float64 0.0 2.0 4.0 6.0 ... 3e+03 4e+03 5e+03
Data variables:
    surf_el            (time, lat, lon) float32 ...
    water_temp         (time, depth, lat, lon) float32 ...
    water_temp_bottom  (time, lat, lon) float32 ...
    salinity           (time, depth, lat, lon) float32 ...
    salinity_bottom    (time, lat, lon) float32 ...
    water_u            (time, depth, lat, lon) float32 ...
    water_u_bottom     (time, lat, lon) float32 ...
    water_v            (time, depth, lat, lon) float32 ...
    water_v_bottom     (time, lat, lon) float32 ...
Attributes:
    classification_level:      UNCLASSIFIED
    distribution_statement:    Approved for public release. Distribution unli...
    downgrade_date:            not applicable
    classification_authority:  not applicable
    institution:               Fleet Numerical Meteorology and Oceanography C...
    source:                    HYCOM archive file
    history:                   archv2ncdf2d
    comment:                   p-grid
    field_type:                instantaneous
    Conventions:               CF-1.6 NAVO_netcdf_v1.1

In [58]:
test.min(), test.max()

(-180.0, 179.9200439453125)

In [63]:
ds["lon"].data

array([ 0.        ,  0.07995605,  0.16003418, ..., -0.23999023,
       -0.16003418, -0.07995605])

In [ ]:
ds = ds.sel(
    lon=slice(extent[0]-1, extent[1]+1),
    lat=slice(extent[2]-1, extent[3]+1)
)